In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import category_encoders as ce

In [ ]:
!pip3 install category_encoders

     |████████████████████████████████| 86 kB 3.0 MB/s 


In [ ]:
data = pd.read_csv('lastfm.csv')
data = data.head(1000)

In [ ]:
print("The shape of the dataframe is: ",data.shape)
data.info()



The shape of the dataframe is:  (1000, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     1000 non-null   int64 
 1   artist   992 non-null    object
 2   sex      997 non-null    object
 3   country  993 non-null    object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [ ]:
print("The column names of the dataframe are: ",data.columns) 
data.describe()

The column names of the dataframe are:  Index(['user', 'artist', 'sex', 'country'], dtype='object')


,user
count,1000.00000
mean,34.36900
std,18.53468
min,1.00000
25%,20.00000
50%,35.00000
75%,47.00000
max,69.00000


In [ ]:
data.isnull().sum() #prints the sum of null values

user       0
artist     8
sex        3
country    7
dtype: int64

In [ ]:
encoder = ce.OrdinalEncoder(cols=['country']) # helps the data to tran
data = encoder.fit_transform(data) 
print(data)

     user                        artist sex  country
0       1         red hot chili peppers   f        1
1       1       the black dahlia murder   f        1
2       1                     goldfrapp   f        1
3       1              dropkick murphys   f        1
4       1                      le tigre   f        1
..    ...                           ...  ..      ...
995    67                sufjan stevens   f        2
996    67                  beastie boys   f        2
997    67  creedence clearwater revival   f        2
998    69                     ghostface   m        2
999    69         explosions in the sky   m        2

[1000 rows x 4 columns]


In [ ]:
data['country']=data['country'].fillna(data['country'].mode()) #filling most probable value for country


In [ ]:
data = data.drop('sex',axis=1) 

In [ ]:
data= data.dropna(subset=['artist'])

In [ ]:
data.head()

,user,artist,country
0,1,red hot chili peppers,1
1,1,the black dahlia murder,1
2,1,goldfrapp,1
3,1,dropkick murphys,1
4,1,le tigre,1


In [ ]:
data.corr()

,user,country
user,1.000000,0.075365
country,0.075365,1.000000


In [ ]:
data = data.drop('country',axis=1)

In [ ]:
columnsname = list(data["artist"].unique())
username = list(data["user"].unique())

In [ ]:
newData=pd.DataFrame(columns=columnsname,index=username)
newData.reset_index(inplace=True) 
newData.head()

,index,red hot chili peppers,the black dahlia murder,goldfrapp,dropkick murphys,le tigre,schandmaul,edguy,jack johnson,eluveitie,...,underworld,joni mitchell,frou frou,dido,spoon,the jimi hendrix experience,elliott smith,peter bjorn and john,my bloody valentine,the new pornographers
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
transactions = []
for i in data['user'].unique():
    transactions.append(list(data[data['user'] == i]['artist'].values))

In [ ]:
x=len(columnsname)
y=len(username)
for i in range(y):
    for j in range(x):
        if columnsname[j] in transactions[i]:
            newData.iloc[i,j]="True"
        else:
            newData.iloc[i,j]="False"

In [ ]:
print(newData)

    index red hot chili peppers the black dahlia murder goldfrapp  \
0    True                  True                    True      True   
1   False                 False                    True     False   
2   False                 False                   False     False   
3   False                 False                   False     False   
4   False                 False                   False     False   
5   False                 False                   False     False   
6   False                 False                   False     False   
7   False                 False                   False     False   
8   False                 False                   False     False   
9   False                 False                   False     False   
10  False                 False                   False     False   
11  False                 False                   False     False   
12  False                 False                   False     False   
13  False                 False   

In [ ]:
newData=newData.applymap(lambda x: True if x == "True" else False)

In [ ]:
frequent_items=apriori(newData,min_support=0.07,use_colnames=True)
print("Total number of frequent items with support more than 0.07 is {}".format(len(frequent_items)))
frequent_items

Total number of frequent items with support more than 0.07 is 67


,support,itemsets
0,0.104167,(index)
1,0.083333,(eluveitie)
2,0.187500,(guano apes)
3,0.083333,(the rolling stones)
4,0.104167,(aphex twin)
...,...,...
62,0.083333,"(n*e*r*d, crystal castles)"
63,0.083333,"(paul mccartney, blur)"
64,0.083333,"(doves, blur)"
65,0.083333,"(paul mccartney, doves)"


In [ ]:
rules=association_rules(frequent_items,metric="confidence",min_threshold=0.3)
print(rules)

                antecedents              consequents  antecedent support  \
0              (guano apes)          (fleetwood mac)            0.187500   
1           (fleetwood mac)             (guano apes)            0.125000   
2              (neil young)             (guano apes)            0.104167   
3              (guano apes)             (neil young)            0.187500   
4             (tenacious d)                    (air)            0.125000   
5                     (air)            (tenacious d)            0.145833   
6             (max richter)            (frank zappa)            0.250000   
7             (frank zappa)            (max richter)            0.104167   
8               (blink-182)            (max richter)            0.104167   
9             (max richter)              (blink-182)            0.250000   
10                (n*e*r*d)        (crystal castles)            0.083333   
11        (crystal castles)                (n*e*r*d)            0.166667   
12         (

In [ ]:
rules.sort_values(by='lift',inplace=True,ascending=False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23,(blur),"(paul mccartney, doves)",0.083333,0.083333,0.083333,1.000000,12.000000,0.076389,inf
22,(doves),"(paul mccartney, blur)",0.083333,0.083333,0.083333,1.000000,12.000000,0.076389,inf
19,"(paul mccartney, blur)",(doves),0.083333,0.083333,0.083333,1.000000,12.000000,0.076389,inf
18,"(paul mccartney, doves)",(blur),0.083333,0.083333,0.083333,1.000000,12.000000,0.076389,inf
15,(blur),(doves),0.083333,0.083333,0.083333,1.000000,12.000000,0.076389,inf
14,(doves),(blur),0.083333,0.083333,0.083333,1.000000,12.000000,0.076389,inf
13,(blur),(paul mccartney),0.083333,0.104167,0.083333,1.000000,9.600000,0.074653,inf
21,(paul mccartney),"(blur, doves)",0.104167,0.083333,0.083333,0.800000,9.600000,0.074653,4.583333
20,"(blur, doves)",(paul mccartney),0.083333,0.104167,0.083333,1.000000,9.600000,0.074653,inf
17,(doves),(paul mccartney),0.083333,0.104167,0.083333,1.000000,9.600000,0.074653,inf


In [ ]:
rules[ (rules['lift'] >= 6) & (rules['confidence'] >= 0.8) ].sort_values(['confidence','lift'],ascending=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
14,(doves),(blur),0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf
15,(blur),(doves),0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf
18,"(paul mccartney, doves)",(blur),0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf
19,"(paul mccartney, blur)",(doves),0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf
22,(doves),"(paul mccartney, blur)",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf
23,(blur),"(paul mccartney, doves)",0.083333,0.083333,0.083333,1.0,12.0,0.076389,inf
13,(blur),(paul mccartney),0.083333,0.104167,0.083333,1.0,9.6,0.074653,inf
17,(doves),(paul mccartney),0.083333,0.104167,0.083333,1.0,9.6,0.074653,inf
20,"(blur, doves)",(paul mccartney),0.083333,0.104167,0.083333,1.0,9.6,0.074653,inf
10,(n*e*r*d),(crystal castles),0.083333,0.166667,0.083333,1.0,6.0,0.069444,inf


Based on the obtained result using the confidence and lift we can infer that the antecedents and consequents occur together